In [87]:
import pandas as pd
import numpy  as np
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lindachen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [57]:

#file = open('../Resources/' + filename, 'r', encoding="ISO-8859-1")
df = pd.read_csv("EA_CSV.csv", engine="python", encoding="UTF-8")

,Names,\nAvailable,M,Tu,W,Th,F,Sa,Su,"TDS topics: DS, ML, Prog, Visu, AI (...)",Expertise,Connect,Title,Unnamed: 13,Our FAQs
0,Amber Teng,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,NaN,https://www.linkedin.com/in/angelavteng/,DS | Quantitative Risk Modeling | Machine Lear...,NaN,NaN
1,Lowri Williams,x,x,NaN,NaN,NaN,NaN,NaN,NaN,yes,"Python, NLP",https://www.linkedin.com/in/lowriwilliams/,Research Data Scientist at Cardiff University,NaN,NaN
2,Sophie Mann,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,"cognitive science/neuroscience, healthcare, ed...",https://www.linkedin.com/in/sophie-maya-mann/,Data Manager at Tennessee Department of Education,NaN,NaN
3,Jingles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes (no programming),in heath sector (eye),https://www.linkedin.com/in/jingles/,PhD Student | Alibaba Group | Healthcare (Eye ...,NaN,NaN
4,Gitansh Khirbat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,"NLP, Statistics, Optimizations",https://www.linkedin.com/in/gkhirbat/,Machine Learning Engineer | Applied Data Science,NaN,NaN


In [80]:
#turn the first row into column
def get_data(df):
    df = df[['Names', 'Expertise', 'Title']]
    #combine the description
    df['description'] = df['Expertise'].fillna('') + " " + df['Title'].fillna('')
    return df
df = get_data(df)

In [89]:
df

,Names,Expertise,Title,description,cleaned,vectors,similarity
0,Amber Teng,NaN,DS | Quantitative Risk Modeling | Machine Lear...,ds quantitative risk modeling machine learning,ds quantitative risk modeling machine learning,"[[-0.17723532, -0.2903112, 0.068666, -0.033575...",0.568061
1,Lowri Williams,"Python, NLP",Research Data Scientist at Cardiff University,python nlp research data scientist cardiff uni...,python nlp research data scientist cardiff uni...,"[[-0.10119242, -0.066452995, 0.05477286, -0.02...",0.649123
2,Sophie Mann,"cognitive science/neuroscience, healthcare, ed...",Data Manager at Tennessee Department of Education,cognitive science neuroscience healthcare educ...,cognitive science neuroscience healthcare educ...,"[[-0.058894068, -0.1063567, 0.007533, -0.01146...",0.671650
3,Jingles,in heath sector (eye),PhD Student | Alibaba Group | Healthcare (Eye ...,heath sector eye phd student alibaba group hea...,heath sector eye phd student alibaba group hea...,"[[0.16218016, -0.117091544, -0.045649227, -0.1...",0.552522
4,Gitansh Khirbat,"NLP, Statistics, Optimizations",Machine Learning Engineer | Applied Data Science,nlp statistics optimizations machine learning ...,nlp statistics optimizations machine learning ...,"[[-0.003264999, -0.19933838, -0.01587975, -0.1...",0.719830
5,Abdullah Farouk,deep learning,Data Scientist at LoyaltyOne,deep learning data scientist loyaltyone,deep learning data scientist loyaltyone,"[[-0.107093, -0.08525175, 0.0056575015, 0.0230...",0.709291
6,Andrew DeCotiis-Mauro,NaN,Software Developer | ETL | Analytics,software developer etl analytics,software developer etl analytics,"[[-0.0781675, 0.02055775, -0.20060076, -0.1636...",0.511581
7,Anton Muehlemann,"math, deep learning (computer vision and NLP)",Mathematics PhD with specialization in non-con...,math deep learning computer vision nlp mathema...,math deep learning computer vision nlp mathema...,"[[-0.07997999, -0.12043766, 0.07192958, 0.0273...",0.600574
8,Arman Didandeh,"NLP, Visual Analytics, Interaction design, Hum...",Data Science + HCI (cloud developer),nlp visual analytics interaction design human ...,nlp visual analytics interaction design human ...,"[[0.012491534, -0.08915531, -0.14028062, -0.05...",0.701345
9,Carlos Mougan,"explainability, xAI, fairness, ethics, shap, lime",Statistician - Data Scientist Trainee at Europ...,explainability xai fairness ethics shap lime s...,explainability xai fairness ethics shap lime s...,"[[-0.15719499, -0.10014918, -0.07136985, 0.078...",0.637793


In [63]:
def clean_text(s):
    #removeNonAscii
    s = "".join(i for i in s if ord(i)<128)

    #return all lower cases
    s = s.lower()

    #remove stop wrods
    s = s.split()
    stops = set(stopwords.words("english"))
    text = [w for w in s if not w in stops]
    text = " ".join(text)

    #remove html
    html_pattern = re.compile('<.*?>')
    text = html_pattern.sub(r'', text)

    #remove punctuation
    text = re.sub(r'[^\w\s]'," ",text)
    return text



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lindachen/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [64]:
def glove_model(file_path='glove.twitter.27B.200d.txt'):
  """
  input: the access to the file which contains the pre-trained glove model
  ouput: a trained model using glove
  """

  embeddings_index = {}
  f = open(file_path, encoding='utf-8')
  for line in f:
      values = line.split()
      word = values[0]
      coefs = np.asarray(values[1:], dtype='float32')
      embeddings_index[word] = coefs
  f.close()
  return embeddings_index

In [65]:
def glove2vec(text, model):
    """
    input: 
    - text: the string that you wanted to turn into vectors
    - model: the glove pre-trained model that you wanted to use

    output:
    an average vector of your input text using the glove model

    """
    
    # Creating a list for storing the vectors (description into vectors)
    global word_embeddings
    word_embeddings = []

    vectors = {}

    avgword2vec = None
    count = 0

    for word in text.split():
        if word in model:
            count += 1
            if avgword2vec is None:
                avgword2vec = model[word]
            else:
                avgword2vec = avgword2vec + model[word]
            
    if avgword2vec is not None:
        avgword2vec = avgword2vec / count
    
    word_embeddings.append(avgword2vec)

    return word_embeddings

In [85]:
def recommendations_glove(input, df=df, col='description'):
    """
    input: 
    - input: key words relating to the article
    - df: the dataframe that we are using
    - col: the column name that contains description of EAs
    - file path: the file path to the pre-trained glove model

    output:
    a list of EAs' names ranked by the most recommended / most suitable EA
    to the least
    """

    #get the model (it is too slow to compute the model every time)
    model = glove_model(file_path='glove.twitter.27B.200d.txt')

    # vecterizd the input
    vector_input = glove2vec(input, model)

    # vecterized all the descriptions of EAs
    df[col] = df[col].apply(clean_text)
    EAs_vectors = [glove2vec(EA, model) for EA in df[col]]

    #drop an EA if that EA has "None" vector
    df['vectors'] = EAs_vectors
    for i, n in enumerate(df['vectors']):
      if n[0] is None:
        df = df.drop([i+1], axis=0)


    # finding cosine similarity for the vectors
    similarity = []
    for n in df['vectors']:
      scores = cosine_similarity(vector_input, n)[0][0]
      similarity.append(scores)
    
    df['similarity'] = similarity


    # sort and find the recommended movie
    df = df.sort_values(by=['similarity'], ascending=False)
    #res_df = df.iloc[:m]


    return df['Names']

In [88]:
input = 'data collection, data ethics issues/initial assumptions'
res = recommendations_glove(input=input, df=df, col='description')

print(res)

26            Carlos Ortega
23        Sergio Betancourt
27           Julia Nikulski
20             Rohan Joseph
10               David Chen
4           Gitansh Khirbat
17              Mael Fabien
15             Hessie Jones
5           Abdullah Farouk
12          Francisco Nunes
8            Arman Didandeh
16             Lester Leong
19      Pier Paolo Ippolito
2               Sophie Mann
25        François St-Amant
1            Lowri Williams
9             Carlos Mougan
21            John Jagtiani
13         Sara A. Metwalli
31               Agni Kumar
22                 Rui Geng
7          Anton Muehlemann
14         Gerasimos Plegas
32                Dean Deng
0                Amber Teng
3                   Jingles
24             Sohaib Ahmad
11    Dimitris Panagopoulos
29          Purvanshi Mehta
6     Andrew DeCotiis-Mauro
30         Ashwin Hariharan
18            Manish Sharma
28         Marin Vlastelica
Name: Names, dtype: object
